### 框架干的最厉害的一件事就是帮我们把返向传播全部计算好了

In [1]:
import torch 

需要求导的，可以手动定义：

In [2]:
#方法1
x = torch.randn(3,4,requires_grad=True)#构建3行4列的矩阵   requires_grad=True表示可以对当前的X进行求导，默认为false
x

tensor([[-0.7139,  0.7866,  0.9592,  1.8350],
        [ 0.2940, -0.1284, -0.3787, -0.9262],
        [ 0.3364, -0.1694, -2.1813,  0.7998]], requires_grad=True)

In [3]:
#方法2
x = torch.randn(3,4)#
x.requires_grad=True
x

tensor([[ 0.6960,  0.2854, -1.0145, -0.1862],
        [-0.4473,  0.7948,  0.1543, -0.4545],
        [-0.3945,  1.2609,  0.4560,  0.3682]], requires_grad=True)

In [4]:
b = torch.randn(3,4,requires_grad=True)

In [5]:
t = x + b

In [6]:
y = t.sum()
y#y当作损失函数，反向传播就是从损失函数开始进行逐层求导

tensor(2.1753, grad_fn=<SumBackward0>)

In [7]:
y.backward()

In [8]:
b.grad

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

虽然没有指定t的requires_grad但是需要用到它，也会默认的

In [9]:
x.requires_grad, b.requires_grad, t.requires_grad

(True, True, True)

### 举个例子看一下：

![title](./img/2.png)

In [10]:
#计算流程
#对x w b随机值的初始化
x = torch.rand(1)
b = torch.rand(1, requires_grad = True)
w = torch.rand(1, requires_grad = True)
y = w * x 
z = y + b 

In [11]:
x.requires_grad, b.requires_grad, w.requires_grad, y.requires_grad#注意y也是需要的

(False, True, True, True)

In [12]:
x.is_leaf, w.is_leaf, b.is_leaf, y.is_leaf, z.is_leaf#判断是否为leaf节点

(True, True, True, False, False)

返向传播计算

In [13]:
z.backward(retain_graph=True)#在pytorch框架中 ，如果不清空梯度会累加起来

In [14]:
w.grad

tensor([0.3698])

In [15]:
b.grad

tensor([1.])

### 做一个线性回归试试水

构造一组输入数据X和其对应的标签y

In [17]:
import numpy as np
x_values = [i for i in range(11)]
x_train = np.array(x_values, dtype=np.float32)#x现在是ndarry的格式还不能输入到pytorch中进行训练 要把ndarry转化为tensor格式
x_train = x_train.reshape(-1, 1)#为了防止后续出错转化为矩阵的格式
x_train.shape

(11, 1)

In [18]:
y_values = [2*i + 1 for i in x_values]
y_train = np.array(y_values, dtype=np.float32)
y_train = y_train.reshape(-1, 1)
y_train.shape

(11, 1)

In [19]:
import torch
import torch.nn as nn

### 线性回归模型
- 其实线性回归就是一个不加激活函数的全连接层

In [20]:
class LinearRegressionModel(nn.Module):#不论构建多么复杂的模型 先把模型类定义出来  继承现有的nn.Module模块
    def __init__(self, input_dim, output_dim):#在构造函数中写用到了那些层
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)  #调用nn的全连接层，传入输入和输出层的维度

    def forward(self, x):#在前向传播中指定要用的层
        out = self.linear(x)
        return out

In [21]:
input_dim = 1
output_dim = 1

model = LinearRegressionModel(input_dim, output_dim)

In [22]:
model

LinearRegressionModel(
  (linear): Linear(in_features=1, out_features=1, bias=True)
)

### 指定好参数和损失函数进行训练

In [23]:
epochs = 1000#循环次数
learning_rate = 0.01#学习率
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)#定义优化器-SGD  （要优化的参数，学习率）
criterion = nn.MSELoss()#指定MSE损失函数

训练模型

In [24]:
for epoch in range(epochs):
    epoch += 1
    # 注意转行成tensor
    inputs = torch.from_numpy(x_train)
    labels = torch.from_numpy(y_train)

    # 梯度要清零每一次迭代
    optimizer.zero_grad() 

    # 前向传播结果
    outputs = model(inputs)

    # 计算损失
    loss = criterion(outputs, labels)

    # 返向传播
    loss.backward()

    # 更新权重参数
    optimizer.step()
    if epoch % 50 == 0:
        print('epoch {}, loss {}'.format(epoch, loss.item()))

epoch 50, loss 0.019124062731862068
epoch 100, loss 0.01090761087834835
epoch 150, loss 0.006221299059689045
epoch 200, loss 0.003548411652445793
epoch 250, loss 0.0020238589495420456
epoch 300, loss 0.0011543527944013476
epoch 350, loss 0.000658379343803972
epoch 400, loss 0.000375513918697834
epoch 450, loss 0.00021417725656647235
epoch 500, loss 0.00012215929746162146
epoch 550, loss 6.967117224121466e-05
epoch 600, loss 3.97425756091252e-05
epoch 650, loss 2.266592491650954e-05
epoch 700, loss 1.2928460819239262e-05
epoch 750, loss 7.374767392320791e-06
epoch 800, loss 4.206250650895527e-06
epoch 850, loss 2.398967581029865e-06
epoch 900, loss 1.3684858686247026e-06
epoch 950, loss 7.80806885813945e-07
epoch 1000, loss 4.4511330088425893e-07


### 测试模型预测结果

In [25]:
predicted = model(torch.from_numpy(x_train).requires_grad_()).data.numpy()#进行一次前向传播进行预测，把结果转化为ndarry格式,方便画图及pandas做表格时需要用ndarry格式
predicted

array([[ 0.998759 ],
       [ 2.9989378],
       [ 4.9991164],
       [ 6.999295 ],
       [ 8.999475 ],
       [10.999654 ],
       [12.999832 ],
       [15.0000105],
       [17.000189 ],
       [19.000368 ],
       [21.000547 ]], dtype=float32)

### 模型的保存与读取

In [26]:
torch.save(model.state_dict(), 'model.pkl')#保存为字典的格式 把权重参数和偏置保存下来

In [27]:
model.load_state_dict(torch.load('model.pkl'))

<All keys matched successfully>

### 使用GPU进行训练
- 只需要把数据和模型传入到cuda里面就可以了

In [28]:
import torch
import torch.nn as nn
import numpy as np


class LinearRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)  

    def forward(self, x):
        out = self.linear(x)
        return out

input_dim = 1
output_dim = 1

model = LinearRegressionModel(input_dim, output_dim)


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")#如果GPU配置好了用GPU
model.to(device)#把模型传入到cuda里


criterion = nn.MSELoss()


learning_rate = 0.01

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 1000
for epoch in range(epochs):
    epoch += 1
    inputs = torch.from_numpy(x_train).to(device)#把训练数据传入到cuda里
    labels = torch.from_numpy(y_train).to(device)

    optimizer.zero_grad() 

    outputs = model(inputs)

    loss = criterion(outputs, labels)

    loss.backward()

    optimizer.step()

    if epoch % 50 == 0:
        print('epoch {}, loss {}'.format(epoch, loss.item()))

epoch 50, loss 0.05185205861926079
epoch 100, loss 0.02957448549568653
epoch 150, loss 0.016868164762854576
epoch 200, loss 0.00962100364267826
epoch 250, loss 0.005487476009875536
epoch 300, loss 0.0031298540998250246
epoch 350, loss 0.0017851460725069046
epoch 400, loss 0.0010181783000007272
epoch 450, loss 0.0005807341658510268
epoch 500, loss 0.0003312241460662335
epoch 550, loss 0.0001889190316433087
epoch 600, loss 0.00010775127884699032
epoch 650, loss 6.145981751615182e-05
epoch 700, loss 3.505383574520238e-05
epoch 750, loss 1.9994407921331003e-05
epoch 800, loss 1.1404186807340011e-05
epoch 850, loss 6.505466899398016e-06
epoch 900, loss 3.7109186905581737e-06
epoch 950, loss 2.1164087229408324e-06
epoch 1000, loss 1.2071883475073264e-06
